In [38]:
import numpy as np
import pandas as pd
import plotly.express as px
import talib
import datetime as dt
from path import Path

In [56]:
#Graphing Module

import plotly.graph_objects as go
from datetime import datetime

def candles(df,date = None):
    '''
    df : whole dataframe
    date : numeric in form yyyymmdd
    '''
    first = lambda x : list(x)[0]
    last = lambda x : list(x)[-1]
    if date:
        df = df[df['Date']==date]
        x = df['Time']
    else:
        df = df.groupby('Date').agg({"Open":first,"High":'max',"Low" : last,"Close":'max'}).reset_index()
        x = df['Date']
    fig = go.Figure(data=[go.Candlestick(
                    x=x,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])
    return fig

In [57]:
bnf = Path("BANK_NIFTY_data")

In [58]:
bank_df = pd.read_csv(bnf.joinpath("BNF_2010_2020.csv"))

In [32]:
candles(bank_df[bank_df['Date']<=20101231])

In [64]:
def make_candle_patterns(df):
    candle_functions  = []
    df["buy?"] = 0
    for i in dir(talib):
        if i.find("CDL")==0:
            candle_functions.append(i)
    for cf in candle_functions:
        df[cf[3:].lower()] = eval(f'talib.{cf}(df["Open"],df["High"],df["Low"],df["Close"])')
        df['buy?'] += df[cf[3:].lower()]
    return df

In [65]:
candles_bank_df = make_candle_patterns(bank_df)

In [55]:
candles_bank_df.head()

,Instrument,Date,Time,Open,High,Low,Close,engulfing,2CROWS,3BLACKCROWS,...,SPINNINGTOP,STALLEDPATTERN,STICKSANDWICH,TAKURI,TASUKIGAP,THRUSTING,TRISTAR,UNIQUE3RIVER,UPSIDEGAP2CROWS,XSIDEGAP3METHODS
0,BANKNIFTY,20100302,09:01,8826.90,8856.95,8826.90,8839.80,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BANKNIFTY,20100302,09:02,8840.75,8853.90,8832.80,8852.30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BANKNIFTY,20100302,09:03,8855.70,8855.70,8844.65,8846.65,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BANKNIFTY,20100302,09:04,8843.85,8850.70,8843.75,8846.70,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BANKNIFTY,20100302,09:05,8846.40,8846.40,8837.75,8841.60,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
date = 20161115
fig = candles(candles_bank_df,date)
for i,row in candles_bank_df[candles_bank_df["Date"]==date].iterrows():
    if row['buy?'] > 300:
        mkr = dict(symbol='triangle-up-open', size = 12)
        sign = [row["Low"] - row['buy?']/100]
    elif row['buy?']<-100:
        mkr = dict(symbol='triangle-down-open', size = 12)
        sign = [row["High"] - row['buy?']/100]
    else:
        continue
    fig.add_trace(
        go.Scatter(
            x=[row['Time']],
            y= sign ,
            mode="markers+text",
            marker=mkr,
    #         text = 'important',
    #         textposition = 'middle right'

        )
    )


In [101]:
fig.show()